In [4]:
#%pip install beautifulsoup4

In [41]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

- Buscar os links para a tese no Stardog
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

In [42]:
url0 = ('https://bdtd.ibict.br/vufind/Search/Results?lookfor=Slave&type=AllFields&limit=20')
url = ('https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/pt-br.php')
url

'https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/pt-br.php'

In [19]:
import certifi
import ssl
import urllib3

In [49]:
#Fazer requisição e parsear o arquivo html
f = requests.get(url, verify=False).text#, verify=False) 
soup = bs(f, "html.parser")

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [50]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="pt-br" xml:lang="pt-br" xmlns="http://www.w3.org/1999/xhtml">
<head>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TK26HK5H');
  </script>
<!-- End Google Tag Manager -->
<title>Riqueza e escravidão no nordeste paulista: Batatais, 1851-1887</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<meta content="index, follow" name="robots"/>
<meta content="O presente trabalho dedica-se ao estudo da economia e da demografia da escr

In [55]:
#Coletando link para as teses
for doc in soup.find_all('a', href=True):
    if doc['href'].endswith('.pdf'):
        path = doc['href']
        prefix_uni = 'https://www.teses.usp.br'
        link = prefix_uni + path

link

'https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/publico/Garavazo_Juliana_mestrado.pdf'

In [58]:
#fazer download do arquivo
def download_file(url, local_filename):
    # NOTE the stream=True parameter below
    r = requests.get(url, verify=False, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return local_filename

In [63]:
download_file(link, 'garavazo.pdf')

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'garavazo.pdf'

In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details

In [2]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [4]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [5]:
# Query que busca as teses e os seus abstracts. Foi incluído um filtro para buscar apenas 
# os abstract em português, inglês ou em outra língua
query = """
SELECT ?thesis ?abstract (lang(?abstract) AS ?lang) WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/abstract> ?abstract.
  FILTER (lang(?abstract) IN("pt", "en"))
  
}

"""

query2 = """
SELECT ?thesis ?ent  WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <https://schema.org/mentions> ?ent.  
}

"""

query3 = """
SELECT ?place   WHERE {
  ?place a <https://schema.org/Place>. 
}

"""
# FILTER (langMatches(lang(?abstract),"pt"))
# FILTER (langMatches(lang(?abstract),"en"))
# FILTER (lang(?abstract) NOT IN("pt", "en"))
# LIMIT 20

csv_results = conn.select(query, content_type='text/csv')
thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
thesis_abstract


,thesis,abstract,lang
0,tag:stardog:api:en_an_abolicionist_argument_pe...,[pt] O presente trabalho busca fazer uma retom...,pt
1,tag:stardog:api:percepcoes_dos_pesquisadores_d...,NaN,pt
2,tag:stardog:api:com_a_condicao_de_servir_gratu...,A presente pesquisa parte da emancipação de es...,pt
3,tag:stardog:api:entre_as_pessoas_mais_gradas_d...,O objeto central que baliza esta pesquisa é a ...,pt
4,tag:stardog:api:saraiva_dantas_e_cotegipe_baia...,A presente tese investiga o baianismo e suas i...,pt
...,...,...,...
22705,tag:stardog:api:micropropagacao_fotoautotrofic...,The success of mass micropropagation of fruit ...,en
22706,tag:stardog:api:contrastes_e_continuidades_em_...,NaN,pt
22707,tag:stardog:api:assombro_de_olhada_de_bicho_um...,This ethnography has as an objective to compre...,en
22708,tag:stardog:api:o_devirclandestino_em_marcados...,In the dissertation O devir-clandestino em Mar...,en
